# "COVID-19: A Socio-political Virus, but Not Quite How You'd Think"
> "Addressing some of the misconceptions surrounding the politicization of COVID-19."

- toc:true
- branch: master
- badges: true
- comments: true
- author: Jerry Lin
- categories: [covid-19, data-visualization]

**DISCLAIMER:** I lean left in my political ideologies. I respect the CDC and believe masks, while not completely effective, are the easiest way to prevent the spread of respiratory diseases when we are unable to socially distance.

The [most recent polling from Civiqs](https://civiqs.com/results/coronavirus_concern?annotations=true&uncertainty=true&zoomIn=true) (updated Sept. 2) shows a large discrepancy in COVID-19 sentiment, split along party lines:

|Party|Extremely<br>Concerned|Moderately<br>Concerned|A Little<br>Concerned|Not Concerned<br>At All|Unsure|
|---|---|---|---|---|---|
|Democrat|**52%**|35%|10%|3%|<1%|
|Republican|7%|19%|28%|**46%**|<1%|
|Independent|**24%**|30%|22%|24%|<1%|

Because of this, it can be easy for one's own party to blame the other, when the reality is probably more complex.

COVID-19 sentiment also appears to be split across race, as well:

|Race|Extremely<br>Concerned|Moderately<br>Concerned|A Little<br>Concerned|Not Concerned<br>At All|Unsure|
|---|---|---|---|---|---|
|White|22%|28%|22%|**28%**|<1%|
|Black|**58%**|27%|10%|4%|<1%|
|Hispanic|**43%**|29%|15%|14%|<1%|
|Other|**34%**|31%|16%|19%|<1%|

This post will explore various demographic factors and how they relate to up-to-date COVID-19 data.

# About

## Data Sources
- The New York Times [COVID-19 repository](https://github.com/nytimes/covid-19-data)
- The [United States Census](https://www.census.gov/data.html)
- [nomanatim](https://nominatim.openstreetmap.org/) and [polygons](http://polygons.openstreetmap.fr/)
- [github.com/tonmcg](https://github.com/tonmcg) and [RRH Elections](https://rrhelections.com/index.php/2018/02/02/alaska-results-by-county-equivalent-1960-2016/)

## How this data was merged
Refer to my [COVID-19 repository](https://github.com/jydiw/nyt-covid-19-data) for notebooks describing how this data was processed.

In [1]:
#hide
import json
import math
import pickle
import re
from datetime import datetime, timedelta
from pytz import timezone
from time import time

import numpy as np
import numpy.polynomial.polynomial as poly
import pandas as pd

import altair as alt
from altair import datum

alt.data_transformers.enable('data_server');

In [2]:
#hide
def optimize(df):
    '''
    Optimizes the data types in a pandas dataframe.
    '''
    dft = df.copy()
    # converts to datetime if possible
    dft = dft.apply(lambda col:
        pd.to_datetime(col, errors='ignore') if col.dtypes=='object' else col)
    
    # if there are less than half as many unique values as there are rows, convert to category
    for col in dft.select_dtypes(include='object'):
        if len(dft[col].unique()) / len(df[col]) < 0.5:
            dft[col] = dft[col].astype('category')
            
    # downcasts numeric columns if possible
    dft = dft.apply(lambda col: 
        pd.to_numeric(col, downcast='integer') if col.dtypes=='int64' else col)
    dft = dft.apply(lambda col: 
        pd.to_numeric(col, downcast='float') if col.dtypes=='float64' else col)
    
    return dft

In [3]:
#hide
with open('../data/processed/nyt_df.p', 'rb') as f:
    nyt_df = pickle.load(f)
with open('../data/processed/info_df.p', 'rb') as f:
    info_df = pickle.load(f)

In [5]:
#collapse-hide
def column_selector(info_df, columns='none', mask=[], exclude=[]):
    
    # only select from numeric columns
    all_columns = info_df.select_dtypes(include='number').columns.tolist()
    
    # empty container if we don't have a list going already
    if columns is 'none':
        columns = []
    elif columns is 'all':
        return all_columns
    
    # includes all columns that have all elements in mask
    # excludes all columns that have any elements in exclude
    new_columns = all_columns
    if len(mask + exclude) > 0:
        if len(mask) > 0:
            new_columns = list(set([c for c in new_columns if all(m in set(re.findall('[a-z]+', c)) for m in mask)]))
        if len(exclude) > 0:
            new_columns = list(set([c for c in new_columns if all(e not in set(re.findall('[a-z]+', c)) for e in exclude)]))
        columns += new_columns
        
    return sorted(list(set(columns)))


def corr(x, y, w, useweight=True):
    
    # only uses elements that are not nan from both lists
    x_ids = ~np.isnan(x)
    y_ids = ~np.isnan(y)
    ids = x_ids & y_ids
    
    if useweight:
        try:
            [xx, xy], [_, yy] = np.cov(x[ids], y[ids], aweights=w[ids])
        except:
            print(x.name)
            print(y.name)
    else:
        [xx, xy], [_, yy] = np.cov(x[ids], y[ids])
    
    return xy / np.sqrt(xx * yy)


def df_merger(nyt_df, info_df, x_cols=None, y_cols=None, date='latest', weight='tot_pop'):
    
    # make sure x and y are valid
    all_y = nyt_df.columns.tolist()
    for y in y_cols:
        if '_per_100k' in y:
            y_cols.append(y.replace('_per_100k', ''))
    y_cols = sorted(list(set([y for y in y_cols if y in all_y])))
    
    all_x = info_df.columns.tolist()
    x_cols = sorted(list(set([c for c in x_cols if c in all_x])))
    
    ## only process specific date and y_cols
    left_columns = list(set(['date', 'fips'] + y_cols))
    if date=='latest':
        left_df = nyt_df[nyt_df['date']==nyt_df['date'].max()][left_columns]
    elif date=='all':
        left_df = nyt_df[left_columns]
    else:
        left_df = nyt_df[nyt_df['date']==date][left_columns]

    ## only process specific x_cols
    right_columns = list(set(['fips', 'state', 'county', weight] + x_cols))
    right_df = info_df[right_columns]
    
    # https://stackoverflow.com/a/47118728/14083095
    # fills nyt_df with entries for counties that do not log cases
    # for more accurate aggregate per capita calculations
    mux = pd.MultiIndex.from_product([left_df['date'].unique(), right_df['fips'].unique()], names=('date', 'fips'))
    left_df = left_df.set_index(['date','fips']).reindex(mux).swaplevel(0,1).reset_index().fillna(0)
   
    df = left_df.merge(right_df, on='fips', how='outer', suffixes=('_x', ''))
    df = df.drop([x for x in df.columns if x[-2:]=='_x'], axis=1)
    
    return df


def make_correlation_table(
    nyt_df, info_df, x_cols=None, y_cols=None,
    date='latest', useweight=True, weight='tot_pop',
    threshold=0.4
):
    
    df = df_merger(nyt_df, info_df, x_cols, y_cols, date, weight)
    
    wct = pd.DataFrame(index=x_cols, columns=y_cols)
    
    for y in y_cols:
        for x in x_cols:
            wct.loc[x, y] = corr(df[y], df[x], df[weight])
    
    wct = wct[(wct >= threshold) | (wct <= -1 * threshold)].dropna()
    
    return wct.sort_values(by=y_cols[0], ascending=False)


def make_correlation_heatmap(
    nyt_df, info_df, date='latest', x_cols=None,
    y_cols=[
        'cases_per_100k', 
        'new_cases_per_100k_15d',
        'delta_new_cases_per_100k_15d',
        'deaths_per_100k',
        'new_deaths_per_100k_15d',
        'delta_new_deaths_per_100k_15d',
        'mortality_rate',
        'mortality_rate_15d'
    ],
    useweight=True, weight='tot_pop', size=50, print_corr=True,
    threshold=0.4
):
    
    df = df_merger(nyt_df, info_df, x_cols, y_cols, date, weight)

    # build weighted correlation matrix from df
    wcm_cols = x_cols + y_cols
    
    wcm = pd.DataFrame(index=x_cols, columns=wcm_cols)
    
    for y in wcm_cols:
        for x in x_cols:
            wcm.loc[x, y] = corr(df[x], df[y], df[weight])
    
    wcm = (wcm.reset_index().rename(columns={'index':'y_feature'}).dropna()
              .melt('y_feature', var_name='x_feature', value_name='corr'))
    wcm['corr'] = np.round(wcm['corr'].astype(float), 4)

    if print_corr:
        print('positive correlations')
        print(
            wcm[(wcm['corr'] >= threshold) & (wcm['corr'] != 1)]
            .sort_values(by=['corr', 'y_feature']).iloc[::2, :]
            .sort_values(by=['y_feature', 'x_feature'])
        )
        print('\nnegative correlations')
        print(
            wcm[(wcm['corr'] <= -1 * threshold) & (wcm['corr'] != -1)]
            .sort_values(by=['corr', 'y_feature']).iloc[::2, :]
            .sort_values(by=['y_feature', 'x_feature'])
        )
    
    # build altair chart
    base = alt.Chart(wcm).encode(
        alt.X(
            'x_feature:O',
            sort=x_cols
        ),
        alt.Y(
            'y_feature:O',
#             sort=columns
        )
    )
    heatmap = base.mark_rect().encode(
        color=alt.Color(
            'corr:Q',
            scale=alt.Scale(
                scheme='redblue',
                domain=[-1, 0, 1]
            )
        ),
        tooltip=[
            alt.Tooltip('x_feature:O'),
            alt.Tooltip('y_feature:O'),
            alt.Tooltip('corr:Q', title='correlation')
        ]
    )
    
    # text
    text = base.mark_text(baseline='middle').encode(
        text=alt.Text('corr:Q',format='.2f'),
        color=alt.condition(
            np.abs(alt.datum.corr) <= 0.5,
            alt.value('black'),
            alt.value('white')
        )
    )
    
    return (heatmap + text).configure_view(step=size)

# Key Points

- While the media tends to focus on anti-lockdown protests, which are largely comprised of White Republicans, those counties have only very recently (mid-August) experienced a higher number of COVID cases per capita.
- The first COVID wave disproportionately affected counties with a high population density (notably New York City). However, counties with higher Black and Hispanic populations are now experiencing a higher number of COVID cases per capita--which may contribute to their higher degree of concern for the virus.

It is a safe assumption that population density (`pop_density`) ought to be most positively correlated to COVID-19 cases per capita. However, below is a table showing the top correlations (weighted by county population) with new COVID-19 cases in the last 15 days.

In [18]:
#collapse-hide
columns = column_selector(info_df, 'none', exclude=['male', 'female', 'tot', 'never', 'rarely', 'sometimes', 'frequently', 'always', 'bachelors', 'graduate'])
make_correlation_table(nyt_df, info_df, x_cols=columns, y_cols=['new_cases_per_100k_15d'], threshold=0.1)

,new_cases_per_100k_15d,new_cases_15d
per_edu_hispanic_nohs,0.19295,0.19811
per_edu_white_nohs,0.192948,-0.22322
per_edu_other_nohs,0.143057,0.112029
per_gop,0.135533,-0.419843
per_pop_hispanic,0.13355,0.574996
per_edu_black_nohs,0.11599,-0.133431
lon,-0.103768,-0.305254
age_pop_hispanic,-0.120035,0.299972
per_pop_asian,-0.126569,0.351137
edu_twoplus,-0.139107,0.1884


Generalizing the above table, we find the following factors to be most positively correlated:

- percent of hispanic, white, other, and black population without a HS diploma
- percent of two-party voters who voted GOP in the 2016 general election
- percent population hispanic

...and the following the most negatively correlated:

- educational attainment
- percent population asian or white
- percent of eligible voters who voted in the 2016 general election
- population density
- median age
- mask discipline
- latitude

Contrary to our initial assumption, dense counties seem to be doing better than their less dense counterparts with regards to recent COVID-19 cases.

Counties which feature a larger proportion of uneducated white and hispanic residents seem to be suffering from a higher concentration of COVID cases--at least in the last 15 days.
> Important: Correlations do not imply causation.

In [19]:
#collapse-hide
columns = column_selector(
    info_df, 
    ['per_gop', 'mask', 'edu', 'median_income', 'age_pop', 'pop_density'], 
    mask=['per', 'pop'], 
    exclude=['male', 'female', 'tot']
)
make_correlation_heatmap(nyt_df, info_df, x_cols=columns, y_cols=['cases_per_100k', 'new_cases_per_100k_15d'], size=50)

positive correlations
            y_feature         x_feature    corr
40                edu     median_income  0.7337
66                edu     per_pop_asian  0.4250
41               mask     median_income  0.4294
67               mask     per_pop_asian  0.4586
93               mask  per_pop_hispanic  0.4599
147           per_gop     per_pop_white  0.6777
44      per_pop_asian     median_income  0.6062
135     per_pop_asian   per_pop_twoplus  0.4405
176  per_pop_hispanic    cases_per_100k  0.4958
215  per_pop_hispanic     new_cases_15d  0.5750
139   per_pop_pacific   per_pop_twoplus  0.8461

negative correlations
            y_feature       x_feature    corr
53                edu         per_gop -0.4200
54               mask         per_gop -0.6443
212           per_gop   new_cases_15d -0.4198
160           per_gop     pop_density -0.4707
57      per_pop_asian         per_gop -0.5709
149     per_pop_black   per_pop_white -0.4284
150  per_pop_hispanic   per_pop_white -0.7645
206     per

alt.LayerChart(...)

First, let's discuss features that not quite independent from each other:

selected positive correlations (> 0.4):
- educational attainment and median income
- educational attainment and percent asian
- mask discipline and median income
- mask discipline and percent asian
- mask discipline and percent hispanic
- median income and percent asian
- percent GOP and percent white

selected negative correlations (< -0.4):
- educational attainment and percent GOP
- mask discipline and percent GOP
- mask discipline and percent white
- percent Asian and percent GOP
- population density and percent GOP

Since there seems to be multicollinearity, we can't simply throw our data into a multiple linear regression.

# Halving the Country: Measuring COVID-19 Cases along Demographic Features

In [29]:
#collapse-hide

def df_splitter(info_df, split_on, splits=2, equal_pop=True, mode='verbose'):
    
    if mode not in ['verbose', 'mean', 'percentile']:
        mode = verbose
        
    info_df = info_df[~info_df[split_on].isna()].sort_values(by=split_on)
    
    if equal_pop:
        # https://stackoverflow.com/a/31871770/14083095
        # splitting df into approx equal populations
        info_df['pop_cumsum'] = info_df['tot_pop'].cumsum()
        subpop = info_df['pop_cumsum'].max() / splits
        info_df['split'] = (info_df['pop_cumsum'] / subpop).apply(math.ceil)
    else:
        # splitting df into approx equal shapes
        info_df['split'] = pd.qcut(info_df[split_on], splits)
        
    replace_dict = {}
    to_replace = info_df['split'].unique()
    
    # renaming our splits into something more readable
    for i, s in enumerate(to_replace):
        if mode == 'verbose':
            replace_dict[s] = f"[{info_df.loc[info_df['split']==s,split_on].min():.2f},"\
            f" {info_df.loc[info_df['split']==s,split_on].max():.2f}]"
        elif mode == 'mean':
            replace_dict[s] = np.round(
                info_df.loc[info_df['split']==s,split_on].mean(),
                decimals=3
            )
        else:
            replace_dict[s] = (100/splits) * (int(i)+1)
    info_df['split'] = info_df['split'].replace(replace_dict)
    
    return info_df

def make_line_timeseries(
    nyt_df, info_df, y='new_cases_per_100k_15sg', splits=2, split_on=None, 
    equal_pop=True
):
    
    y_title = split_on
    y_subtitle = 'county'
    if equal_pop:
        y_subtitle = 'pop'
    
    # check number of splits and only split on numeric columns
    # otherwise, use names as the different lines (setting splits=1)
    splits = int(splits)
    if split_on in info_df.select_dtypes(exclude='number').columns:
        splits = 1
    y_ = [y]
    if '_per_100k' in y:
        y_ = [y.replace('_per_100k', '')]
    elif y is 'mortality_rate':
        y_ = ['cases', 'deaths']
    # first split df so that we can plot different lines
    if splits > 1:
        info_df = df_splitter(info_df, split_on, splits, equal_pop)
        merged = df_merger(
            nyt_df, info_df, x_cols=[split_on, 'split'], y_cols=y_, date='all',
            weight='tot_pop'
        )
        # 'split' column generated by df_splitter()
        split_on = 'split'
    else:
        merged = df_merger(
            nyt_df, info_df, x_cols=[split_on], y_cols=y_, date='all'
        )
        
    # recalculate aggregates
    if '_per_100k' in y:
        y_ = y.replace('_per_100k', '')
        data = merged.groupby(by=['date', split_on])[y_].sum().fillna(0)\
               / merged.groupby(by=['date', split_on])['tot_pop'].sum() * 100_000
    elif y is 'mortality_rate':
        data = merged.groupby(by=['date', split_on])['deaths'].sum()\
               / merged.groupby(by=['date', split_on])['cases'].sum()

#     elif y is 'mortality_rate':
#         data = merged.groupby(by=['date', split_on])[y_].sum().fillna(0)\
#                / merged.groupby(by=['date', split_on])['tot_pop'].sum()
    else:
        data = merged.groupby(by=['date', split_on])[y].sum().fillna(0)
    data = data.reset_index().rename(columns={0: y})
    
    
    # nearest point selection
    nearest = alt.selection(type='single', nearest=True, on='mouseover',
                            fields=['date'], empty='none')
    
    # base line chart
    lines = alt.Chart(data).mark_line().encode(
        x='date:T',
        y=alt.Y(
            f'{y}:Q',
            title=y.replace('_', ' ')
        ),
        color=f'{split_on}:N'
    )
    
    # selects nearest points based on date
    selectors = alt.Chart(data).mark_point().encode(
        x='date:T',
        opacity=alt.value(0)
    ).add_selection(nearest)
    
    # marks a point on line where selected
    points = lines.mark_point().encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0))
    )
    
    # white background for text
    white_text = lines.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=3).encode(
        text=alt.condition(nearest, f'{y}:Q', alt.value(' '), format='.1f')
    )
    
    # text showing y value
    text = lines.mark_text(align='left', dx=5, dy=-5).encode(
        text=alt.condition(nearest, f'{y}:Q', alt.value(' '), format='.1f')
    )
    
    # rule showing nearest selector
    rules = alt.Chart(data).mark_rule(color='gray').encode(
        x='date:T',
        size=alt.value(1)
    ).transform_filter(nearest)
    
    return alt.layer(
        lines, selectors, points, rules, white_text, text
    ).configure_axis(
        gridDash=[1,2]
    ).properties(
        width=540, height=320,
        title=f'{y} vs. {y_title}'
    )

Dense counties (notably New York City) suffered high COVID cases early on. Up until about mid-July, however, lower-density counties have had a higher proportion of COVID cases.

In [30]:
#collapse-hide
make_line_timeseries(
    nyt_df, 
    info_df, 
    y='new_cases_per_100k_15sg', 
    splits=2,
    split_on='pop_density', 
    equal_pop=True)

alt.LayerChart(...)

Republican-leaning counties have also only very recently had higher COVID cases than their Democrat-leaning counterparts. Note that there is a considerable amount of collinearity between population density and percent GOP (-0.47 correlation).

In [31]:
#collapse-hide
make_line_timeseries(
    nyt_df, 
    info_df, 
    y='new_cases_per_100k_15sg', 
    splits=2,
    split_on='per_gop', 
    equal_pop=True)

alt.LayerChart(...)

Counties with a higher white population have been doing relatively well with COVID (up until now).

In [32]:
#collapse-hide
make_line_timeseries(
    nyt_df, 
    info_df, 
    y='new_cases_per_100k_15sg', 
    splits=2,
    split_on='per_pop_white', 
    equal_pop=True)

alt.LayerChart(...)

Counties with higher Black and Hispanic populations have been disproportionately affected by COVID as well. Recall the COVID sentiment survey: White respondents are, on average, less concerned with COVID than Black and Hispanic respondents. Perhaps this has to do with them not being as negatively affected by the virus.

In [33]:
#collapse-hide
make_line_timeseries(
    nyt_df, 
    info_df, 
    y='new_cases_per_100k_15sg', 
    splits=2,
    split_on='per_pop_black', 
    equal_pop=True)

alt.LayerChart(...)

In [34]:
#collapse-hide
make_line_timeseries(
    nyt_df, 
    info_df, 
    y='new_cases_per_100k_15sg', 
    splits=2,
    split_on='per_pop_hispanic', 
    equal_pop=True)

alt.LayerChart(...)

Counties with lower educational attainment are now experiencing a disproportionate amount of COVID cases.

In [35]:
#collapse-hide
make_line_timeseries(
    nyt_df, 
    info_df, 
    y='new_cases_per_100k_15sg', 
    splits=2,
    split_on='edu', 
    equal_pop=True)

alt.LayerChart(...)

Counties with higher mask discipline are experiencing a lower rate of COVID cases--however only since mid-July.

In [36]:
#collapse-hide
make_line_timeseries(
    nyt_df, 
    info_df, 
    y='new_cases_per_100k_15sg', 
    splits=2,
    split_on='mask', 
    equal_pop=True)

alt.LayerChart(...)

Counties with higher median income also have a lower rate of COVID cases.

In [37]:
#collapse-hide
make_line_timeseries(
    nyt_df, 
    info_df, 
    y='new_cases_per_100k_15sg', 
    splits=2,
    split_on='median_income', 
    equal_pop=True)

alt.LayerChart(...)

# Visualizing Via Heatmap

If we want to gain more resolution by having more splits, the line chart can get a little difficult to read. Therefore, we can visualize those relationships using both a heatmap and corresponding bar chart for clarity.

In [39]:
#collapse-hide
def make_heatmap_timeseries(
    nyt_df, info_df, y='new_cases_per_100k_15sg', splits=10, split_on=None,
    equal_pop=True, mode='percentile'
):
    y_title = split_on
    y_subtitle = 'county'
    if equal_pop:
        y_subtitle = 'pop'
    # check number of splits and only split on numeric columns
    # otherwise, use names as the different lines (setting splits=1)
    splits = int(splits)
    if split_on in info_df.select_dtypes(exclude='number').columns:
        splits = 1
    y_ = [y]
    if '_per_100k' in y:
        y_ = [y.replace('_per_100k', '')]
    elif y is 'mortality_rate':
        y_ = ['cases', 'deaths']
    # first split df so that we can plot different lines
    if splits > 1:
        info_df = df_splitter(info_df, split_on, splits, equal_pop, mode)
        merged = df_merger(
            nyt_df, info_df, x_cols=[split_on, 'split'], y_cols=y_, date='all',
            weight='tot_pop'
        )
        # 'split' column generated by df_splitter()
        split_on = 'split'
    else:
        merged = df_merger(
            nyt_df, info_df, x_cols=[split_on], y_cols=y_, date='all'
        )
        
    # recalculate aggregates
    if '_per_100k' in y:
        y_ = y.replace('_per_100k', '')
        data = merged.groupby(by=['date', split_on])[y_].sum().fillna(0)\
               / merged.groupby(by=['date', split_on])['tot_pop'].sum() * 100_000
    elif y is 'mortality_rate':
        data = merged.groupby(by=['date', split_on])['deaths'].sum()\
               / merged.groupby(by=['date', split_on])['cases'].sum()
    else:
        data = merged.groupby(by=['date', split_on])[y].sum().fillna(0)
    data = data.reset_index().rename(columns={0: y})
    
    y_alt = f'{split_on}:O'
    
    # nearest point selection
    nearest = alt.selection(type='single', nearest=True, on='mouseover',
                            fields=['date'], empty='all')
    
    # title
    dx = 160
    dy = splits*9
    title = alt.Chart(data).mark_text(dx=dx, dy=dy, size=20).encode(
        text='monthdate(date):T'
    ).transform_filter(nearest)
    
    w_title = alt.Chart(data).mark_text(dx=dx, dy=dy, stroke='white', strokeWidth=3, size=20).encode(
        text='monthdate(date):T'
    ).transform_filter(nearest)
    
    # right panel: heatmap
    heatmap = alt.Chart(data).mark_rect().encode(
        alt.X(
            'monthdate(date):T',
            axis=alt.Axis(format='%b %d')
        ),
        alt.Y(
            y_alt,
            sort=alt.EncodingSortField(f'{split_on}', order='descending'),
            title=f'{y_title} ({y_subtitle} {mode})'
        ),
        color=alt.Color(
            f'{y}:Q',
            scale=alt.Scale(
                scheme='lightmulti'
            )
        )
    ).add_selection(nearest)
    
    # left panel: bar chart
    bars = alt.Chart(data).mark_bar().encode(
        alt.X(
            f'{y}:Q',
            scale=alt.Scale(
                domain=[0, data[y].max()]
            )
        ),
        alt.Y(
            y_alt,
            sort=alt.EncodingSortField(f'{split_on}', order='descending'),
            title=f'{y_title} ({y_subtitle} {mode})'
        ),
        color=alt.Color(
            f'{y}:Q',
            scale=alt.Scale(
                scheme='lightmulti'
            )
        ),
        tooltip=[
            alt.Tooltip(f'{y}:Q'),
            alt.Tooltip(y_alt),
        ]
    ).transform_filter(nearest)
    
    
    # selects nearest points based on date
    selectors = alt.Chart(data).mark_point().encode(
        x='monthdate(date):T',
        opacity=alt.value(0)
    ).add_selection(nearest)
    
    return (heatmap | bars+w_title+title).properties(
        title=f'{y} vs {y_title}'
    )

We see how dense counties were almost exclusively hit first. However, the second wave seems to be affecting counties of all population densities.

In [47]:
#collapse-hide
make_heatmap_timeseries(nyt_df, info_df, y='new_cases_per_100k_15sg', splits=10, split_on='pop_density', equal_pop=True, mode='percentile')

alt.HConcatChart(...)

We see with more clarity how Republican counties have actually done much better than Democratic ones only up until recently. Again, since population density is collinear with percent GOP, this chart is essentially an inverse of the population density chart above.

In [40]:
#collapse-hide
make_heatmap_timeseries(nyt_df, info_df, y='new_cases_per_100k_15sg', splits=10, split_on='per_gop', equal_pop=True, mode='percentile')

alt.HConcatChart(...)

Mask discipline also has not shown much correlation until recently. Democratic-leaning urban centers tend to have higher mask discipline, which can explain the early cases among higher-percentile counties. However, we see that counties with lower mask discipline are experiencing a higher number of cases.

In [41]:
make_heatmap_timeseries(nyt_df, info_df, y='new_cases_per_100k_15sg', splits=10, split_on='mask', equal_pop=True, mode='percentile')

alt.HConcatChart(...)

# Conclusion

- COVID-19 affected dense counties first
- Later cases have since moved to less dense counties, which happen to be more Republican-leaning and have lower mask discipline.
- Republican counties have only very recently (mid-August) experienced a higher number of COVID cases per capita.